# Regressão 

#### Foi usado uma base de dados de características de casas com o valor delas, para se fazer uma regressão, é um método puramente estatístico. Foi usado as bibliotecas scikit learn( biblioteca de algoritmos de ML e métricas) e pandas (biblioteca de funçĩes para manipulação de base de dados).


In [184]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import linear_model
from sklearn.metrics import r2_score , mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor




rg1 = pd.read_csv('train.csv')

rgY = rg1['SalePrice']

rg1 = rg1.drop(columns=['Id'])

## Pré-processamento

#### Foi utilizado uma função para medir a correlação do Frame, e usado a coluna das correlações com o objetivo que é 'SalePrice', assim foi possível retirar as características com menor correlação com o alvo, deixando a regressão mais precisa e com o menor erro possível.

In [209]:
cor = rg1.corr()

cor_target = cor["SalePrice"]

irrelevant_features = cor_target[cor_target < 0.25]
irrelevant_features.index

Index(['MSSubClass', 'OverallCond', 'BsmtFinSF2', 'BsmtUnfSF', 'LowQualFinSF',
       'BsmtFullBath', 'BsmtHalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

#### Após retirar as características com menor correlação, foi preenchido a característica 'GarageYrBlt', com valores já existentes nessa coluna, pois seria ruim usar a média ou mediana como valor para os dados faltosos, visando que em um cálculo isso pode criar um vies negativo no resultado. Então foi tratado os valores vazios com uma string indicando NaN, isso foi feito pois nessa base, os valores nulos em algumas colunas representam que aquela característica não está presente no imóvel, depois disso foi usado o LabelEncoder para transformar os objetos em valores numéricos, pois seria impossível usar os modelos em strings, e foi normalizado para tentar melhorar os resultados, mas isso foi pouco efetivo nesse caso.

In [210]:
rg = rg1.drop(columns= irrelevant_features.index)

rgX = rg.drop(columns='SalePrice')

rgX['GarageYrBlt'] = rgX['GarageYrBlt'].interpolate(method= 'pad')

rgX = rgX.fillna('NaN')

le = preprocessing.LabelEncoder()

for i in rgX.columns:
	if rgX[i].dtypes == 'object':
		rgX[i] = le.fit_transform(rgX[i].astype(str))
	
# rgX = Normalizer().fit_transform(rgX)


trainX, testX, trainY, testY = train_test_split(rgX,rgY, test_size=0.2)


In [211]:
Bayes = linear_model.BayesianRidge()

Bayes.fit(trainX, trainY)

BayesPredict = Bayes.predict(testX)

print("Bayes Score:" , r2_score(testY, BayesPredict), mean_absolute_error(testY, BayesPredict))


Bayes Score: 0.8626036585216369 19994.537071663464


In [212]:
Linear = linear_model.LinearRegression()

Linear.fit(trainX, trainY)

LinearPredict = Linear.predict(testX)

print("Linear Score:", r2_score(testY, LinearPredict), mean_absolute_error(testY, LinearPredict))


Linear Score: 0.845642506876757 21100.982262789392


In [213]:
Neighbors = KNeighborsRegressor(n_neighbors = 9, weights = 'distance')

Neighbors.fit(trainX, trainY)

NeighborsPredict = Neighbors.predict(testX)

print("KNeighbors Score:", r2_score(testY, NeighborsPredict) , mean_absolute_error(testY, NeighborsPredict))

KNeighbors Score: 0.6885148696780714 29169.724260892785


In [214]:
Tree = DecisionTreeRegressor(max_depth = 7)

Tree.fit(trainX, trainY)

TreePredict = Tree.predict(testX)

print('Tree Score:', r2_score(testY, TreePredict), mean_absolute_error(testY, TreePredict))


Tree Score: 0.7309479357381337 26566.87436571258


In [215]:
Rede = MLPRegressor(learning_rate_init = 0.02,max_iter = 1000, batch_size = 100)

Rede.fit(trainX, trainY)

RedePredict = Rede.predict(testX)

print('Rede Neural Score:', r2_score(testY, RedePredict), mean_absolute_error(testY, RedePredict))

Rede Neural Score: 0.8150859197574687 22846.019706981184


## Resultados

#### Os modelos lineares foram os melhores algoritmos, entre os selecionados, pode-se perceber que eles não só tiveram um menor erro médio como também um coeficiente de determinação maior, isso indica que os modelos se encaixaram melhor nos dados. Isso também se deve pelo fato das características com menor correlação foram tiradas da base, isso influência diretamente nos modelos de regressão linear, deixando-os mais precisos, porém isso não foi observado quando retirado mais valores, a métrica de 25% de correlação foi testada e obteve os melhores resultados, não se pode provar que é a menor média de erros possível, mas nesses testes foram essas duas a melhores: Regressão Linear Bayesiana e Regressão Linear.

In [218]:
frame = []

frame+= [['Regressão Linear Bayesiana',mean_absolute_error(testY, BayesPredict)]]
frame+= [['Regressão Linear',mean_absolute_error(testY, LinearPredict)]]

df = pd.DataFrame(frame, columns = ['Modelo','Média de Erro Absoluto'],)

df


,Modelo,Média de Erro Absoluto
0,Regressão Linear Bayesiana,19994.537072
1,Regressão Linear,21100.982263
